In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(629))
random.seed(40)
random.shuffle(index)

train_index = index[0:440]
val_index = index[440:503]
test_index = index[503:629]

In [3]:
#dataimage_train = [dataimage[j] for j in train_index]
#print(dataimage_train)

In [4]:
lab = np.zeros((629,1)) #first AD:188 then CN:288
lab[228:629, 0] = 1 #MCI -- 1; CN -- 0
for i in range(0, 629):
    print(i, lab[i])

0 [0.]
1 [0.]
2 [0.]
3 [0.]
4 [0.]
5 [0.]
6 [0.]
7 [0.]
8 [0.]
9 [0.]
10 [0.]
11 [0.]
12 [0.]
13 [0.]
14 [0.]
15 [0.]
16 [0.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [0.]
27 [0.]
28 [0.]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [0.]
35 [0.]
36 [0.]
37 [0.]
38 [0.]
39 [0.]
40 [0.]
41 [0.]
42 [0.]
43 [0.]
44 [0.]
45 [0.]
46 [0.]
47 [0.]
48 [0.]
49 [0.]
50 [0.]
51 [0.]
52 [0.]
53 [0.]
54 [0.]
55 [0.]
56 [0.]
57 [0.]
58 [0.]
59 [0.]
60 [0.]
61 [0.]
62 [0.]
63 [0.]
64 [0.]
65 [0.]
66 [0.]
67 [0.]
68 [0.]
69 [0.]
70 [0.]
71 [0.]
72 [0.]
73 [0.]
74 [0.]
75 [0.]
76 [0.]
77 [0.]
78 [0.]
79 [0.]
80 [0.]
81 [0.]
82 [0.]
83 [0.]
84 [0.]
85 [0.]
86 [0.]
87 [0.]
88 [0.]
89 [0.]
90 [0.]
91 [0.]
92 [0.]
93 [0.]
94 [0.]
95 [0.]
96 [0.]
97 [0.]
98 [0.]
99 [0.]
100 [0.]
101 [0.]
102 [0.]
103 [0.]
104 [0.]
105 [0.]
106 [0.]
107 [0.]
108 [0.]
109 [0.]
110 [0.]
111 [0.]
112 [0.]
113 [0.]
114 [0.]
115 [0.]
116 [0.]
117 [0.]
118 [0.]
119 [0.]
120 [0.]
121 [0.]
122 [0.]
123

In [5]:
cn = 0.
mci = 0.
total_num = 0.
for i in lab:
    if(i == 1):
        mci += 1
    elif(i == 0):
        cn += 1
total_num = cn + mci
print(mci, cn, total_num)

401.0 228.0 629.0


In [6]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_CN_MCI_Feb2.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((629,1)) #first AD:188 then CN:288
        self.lab[228:629, 0] = 1 #CN -- 0; MCI -- 1
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            self.img = self.img[train_index]
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            for i in range(440):
                for j in range(3):
                    self.img[i][j] = self.img[i][j] + np.random.normal(0, 0.01, (121, 121))
                    print(self.img[i][j].shape)
                    #count += 1
            #print(count)
            self.lab = self.lab[train_index]
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
       
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 

        return sample


In [7]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=400,
                        shuffle=True, num_workers=4)

print("finished")

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

#initial: 
#121 * 121 * 3(feature map number)
#60 * 60 * 16
#30 * 30 * 32
#15 * 15 * 64
#7 * 7 * 128
#3 * 3 * 256

#Linear: 3 * 3 * 256
#3 * 3 * 256(2304)--->1024 --->256 ---> 64 --->3 
class Net_2D(nn.Module):
    def __init__(self):
        super(Net_2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3 * 3 * 256, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 3 * 3 * 256)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net_2d = Net_2D().cuda()

print("finished")

finished


In [9]:
import torch.optim as optim
import torch 
Weight = torch.tensor([total_num / cn, total_num / mci])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net_2d.parameters(), lr=0.005, momentum=0.9)

print("finished")

finished


In [10]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

loss_val_min = 1000000
threshold = 10
count = 0
count_big = 0
epoch = 50000
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_2d.train()

        # forward + backward + optimize
        outputs = net_2d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_2d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        #print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_2d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        #print(loss.item())
        #print(type(loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    if(total_val_loss >= loss_val_min):
        if(total_val_loss < 0.68):
            count += 1
    else:
        loss_val_min = total_val_loss
        count = 0
        torch.save(net_2d, 'best_model__Feb2_NOISE_weighted_MCI_CN.pkl')
    #if(total_val_loss < min_total_loss):
        #min_total_loss = total_val_loss
        #save best model
        #torch.save(net_2d, 'best_model_Jan27_2d_3types.pkl')
    if(count > threshold):
        break
    if(total_val_loss > 0.7):
        count_big += 1
    if(count_big > threshold):
        break
    if(total_val_loss < 0.67):
        break

print("we're done training")

[1] loss: 0.694
[2] loss: 0.701
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.703




/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net_2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[1] loss: 0.693
[2] loss: 0.708
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.703


[1] loss: 0.695
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.702


[1] loss: 0.693
[2] loss: 0.707
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.702


[1] loss: 0.694
[2] loss: 0.697
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.701


[1] loss: 0.694
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.701


[1] loss: 0.694
[2] loss: 0.687
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.700


[1] loss: 0.694
[2] loss: 0.687
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.700


[1] loss: 0.694
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.700


[1] loss: 0.692
[2] loss: 0.706
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.698
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.694
[2] loss: 0.688
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.694
[2] loss: 0.688
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.692
[2] loss: 0.701
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.694
[2] loss: 0.687
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.692
[2] loss: 0.699
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.689
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.694
[2] loss: 0.682
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.693
[2] loss: 0.695
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.693


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.693
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.689
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.688
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.695
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.693
[2] loss: 0.688
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.696


[1] loss: 0.693
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.698
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.692
[2] loss: 0.700
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.698
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.692
[2] loss: 0.699
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.691
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.697


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.693
[2] loss: 0.691
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.696
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.693
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.690
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.695


[1] loss: 0.693
[2] loss: 0.694
$$Finished Training This Epoch
Start a new val_epoch
Val

In [11]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model__Feb2_NOISE_weighted_MCI_CN.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    #print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    #print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("predicted:\n", results_type, "\nactual:\n", labels)

CN_num = 0
MCI_num = 0
for results in results_type:
    if (results.float() == 1):
        MCI_num += 1
    if (results.float() == 0):
        CN_num += 1
print(MCI_num, CN_num)       
for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        correct_sum += 1

correct_sum_CN = 0
correct_sum_MCI = 0
#correct_sum_MCI = 0

CN_num = 0
#MCI_num = 0
MCI_num = 0
for label in labels:
    if (label.float() == 1):
        MCI_num += 1
    if (label.float() == 0):
        CN_num += 1
    #if (label.float() == 2):
        #MCI_num += 1
print("labels", MCI_num, CN_num)
#print("correct_sum_final", correct_sum)
#print(total_sum)
CN_num = 0
#MCI_num = 0
MCI_num = 0
for results in results_type:
    if (results.float() == 1):
        MCI_num += 1
    if (results.float() == 0):
        CN_num += 1
    #if (results.float() == 2):
        #MCI_num += 1
#print("predicted", AD_num, CN_num, MCI_num)


for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        if(results_type[i].float() == 1):
            correct_sum_MCI += 1
    if(labels[i].float() == results_type[i].float()):
        if(results_type[i].float() == 0):
            correct_sum_CN += 1
    #if(labels[i].float() == results_type[i].float()):
        #if(results_type[i].float() == 2):
            #correct_sum_MCI += 1
    total_sum += 1
    
print("correct_sum", correct_sum)
print("correct_sum_MCI", correct_sum_MCI)
print("correct_sum_CN", correct_sum_CN)
#print("correct_sum_MCI", correct_sum_MCI)
print(total_sum)


predicted:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) 
actual:
 tensor([1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 1